In [1]:
#r "nuget: Microsoft.Extensions.Configuration.Json,7.0.0"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets,7.0.0"
#r "nuget: Microsoft.Extensions.Logging.Console,7.0.0"
#r "nuget: Microsoft.Extensions.Logging.Debug,7.0.0"
#r "nuget: Microsoft.SemanticKernel.Core, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.Connectors.AI.OpenAI, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.Connectors.AI.HuggingFace, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Postgres, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Sqlite, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Weaviate, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.Functions.Grpc, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.Functions.OpenAPI, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.Plugins.Document, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.Planners.Core, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.0.0-beta4"
#r "nuget: Microsoft.SemanticKernel.TemplateEngine.Basic, 1.0.0-beta4"
#r "nuget: System.CommandLine,2.0.0-beta4.22272.1"
#r "nuget: YamlDotNet,13.5.2"

Installed Packages Microsoft.Extensions.Configuration.Json, 7.0.0 Microsoft.Extensions.Configuration.UserSecrets, 7.0.0 Microsoft.Extensions.Logging.Console, 7.0.0 Microsoft.Extensions.Logging.Debug, 7.0.0 Microsoft.SemanticKernel.Abstractions, 1.0.0-beta4 Microsoft.SemanticKernel.Connectors.AI.HuggingFace, 1.0.0-beta4 Microsoft.SemanticKernel.Connectors.AI.OpenAI, 1.0.0-beta4 Microsoft.SemanticKernel.Connectors.Memory.Postgres, 1.0.0-beta4 Microsoft.SemanticKernel.Connectors.Memory.Sqlite, 1.0.0-beta4 Microsoft.SemanticKernel.Connectors.Memory.Weaviate, 1.0.0-beta4 Microsoft.SemanticKernel.Core, 1.0.0-beta4 Microsoft.SemanticKernel.Functions.Grpc, 1.0.0-beta4 Microsoft.SemanticKernel.Functions.OpenAPI, 1.0.0-beta4 Microsoft.SemanticKernel.Planners.Core, 1.0.0-beta4 Microsoft.SemanticKernel.Plugins.Core, 1.0.0-beta4 Microsoft.SemanticKernel.Plugins.Document, 1.0.0-beta4 Microsoft.SemanticKernel.Plugins.Memory, 1.0.0-beta4 Microsoft.SemanticKernel.Plugins.Web, 1.0.0-beta4 Microsoft.SemanticKernel.TemplateEngine.Basic, 1.0.0-beta4 System.CommandLine, 2.0.0-beta4.22272.1 YamlDotNet, 13.5.2

In [4]:
using System;
using System.IO;
using System.CommandLine;
using System.Collections.Immutable;
using System.Text.RegularExpressions;
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.AI.TextCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.TextEmbedding;
using Microsoft.SemanticKernel.Functions.OpenAPI;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.TemplateEngine.Basic;

In [5]:
var notebookDir = Environment.CurrentDirectory;
var parentDir = Directory.GetParent(notebookDir).FullName;

var builder = new ConfigurationBuilder()
    .SetBasePath(parentDir)  // Set the base path
    .AddJsonFile("config/appsettings.json", optional: false, reloadOnChange: true)  // Load the main appsettings.json file
    .AddJsonFile("appsettings.plugins.json", optional: true, reloadOnChange: true);  // Load the environment-specific appsettings file

var conf = builder.Build();

In [6]:
ILoggerFactory loggerFactory = LoggerFactory.Create(builder =>
{
  builder.SetMinimumLevel(LogLevel.Warning)
          .AddConsole()
          .AddDebug();
});

In [7]:
var serviceId = conf.GetValue<string>("serviceId");
var endpointType = conf.GetValue<string>("endpointType");
var serviceType = conf.GetValue<string>("serviceType");
var deploymentOrModelId = conf.GetValue<string>("deploymentOrModelId");
var apiKey = conf.GetValue<string>("apiKey");
var orgId = conf.GetValue<string>("orgId");

// Taken from: https://github.com/microsoft/semantic-kernel/blob/main/dotnet/samples/KernelSyntaxExamples/Example42_KernelBuilder.cs
//
// Kernel.Builder returns a new builder instance, in case you want to configure the builder differently.
// ==========================================================================================================
// Kernel instances can be created the usual way with "new", though the process requires particular
// attention to how dependencies are wired together. Although the building blocks are available
// to enable custom configurations, we highly recommend using KernelBuilder instead, to ensure
// a correct dependency injection.
// ----------------------------------------------------------------------------------------------------------
IKernel kernel = Kernel.Builder
                     .WithLoggerFactory(loggerFactory)
                     .WithOpenAITextCompletionService(
                            modelId: deploymentOrModelId,
                            apiKey: apiKey,
                            orgId: orgId,
                            serviceId: serviceId)
                    .Build();

In [8]:
var rootDirectory = conf.GetSection("PluginSettings:Root").Get<string>();
var pluginDirectories = conf.GetSection("PluginSettings:Plugins").Get<string[]>();

var pluginsRoot = Path.Combine(parentDir, rootDirectory!);
var pluginImport = kernel.ImportSemanticFunctionsFromDirectory(pluginsRoot, pluginDirectories!);

In [9]:
int nameMaxLength = pluginImport.Max(p => p.Value.Name.Length) + 2;  // +2 for padding
int descMaxLength = pluginImport.Max(p => p.Value.Description.Length) + 2;

// Header
Console.WriteLine($"+{new string('-', nameMaxLength)}+{new string('-', descMaxLength)}+");
Console.WriteLine($"|{"Plugin".PadRight(nameMaxLength)}|{"Description".PadRight(descMaxLength)}|");
Console.WriteLine($"+{new string('-', nameMaxLength)}+{new string('-', descMaxLength)}+");

// Rows
foreach (var p in pluginImport)
{
    Console.WriteLine($"|{p.Value.Name.PadRight(nameMaxLength)}|{p.Value.Description.PadRight(descMaxLength)}|");
}

// Footer
Console.WriteLine($"+{new string('-', nameMaxLength)}+{new string('-', descMaxLength)}+");


+---------------+----------------------------------------------------+
|Plugin         |Description                                         |
+---------------+----------------------------------------------------+
|Chat           |Answer user questions                               |
|Helm           |Create Helm Charts for given devops task            |
|Kubernetes     |Create kubernetes YAML files for given devops task  |
|CSharp         |Create .NET projects for given task                 |
|TypeScript     |Create TypeScript source codes for given task       |
|CreateHtmlDoc  |Create HTML documents                               |
|ExtractJS      |Extract JavaScript from an HTML document            |
|UrlFinder      |Find the URL of a web page                          |
+---------------+----------------------------------------------------+


In [10]:
string description = "create a kubernetes secret YAML for user/password";
var context = new ContextVariables();

string key = "input";
context.Set(key, description);

var result = await kernel.RunAsync(context, pluginImport["Kubernetes"]);
Console.WriteLine(result.GetValue<string>());


FILE: secret.yaml
#----#
apiVersion: v1
kind: Secret
metadata:
  name: my-secret
type: Opaque
data:
  username: dXNlcg==
  password: cGFzc3dvcmQ=
#----#
